# Проект "Рекомендация тарифов"



В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.
Необходимо построить модель с максимально большим значением accuracy. Необходимо довести долю правильных ответов по крайней мере до 0.75. Проверить accuracy на тестовой выборке самостоятельно.

Инструкция по выполнению проекта

**Шаг 1. Открыть файл с данными и изучите его. Путь к файлу: /datasets/users_behavior.csv.**

**Шаг 2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.**

**Шаг 3. Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.**

**Шаг 4. Проверить качество модели на тестовой выборке.**

**Шаг 5. Дополнительное задание: проверить модели на вменяемость.**



**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков,

`minutes` — суммарная длительность звонков в минутах,

`messages` — количество sms-сообщений,

`mb_used` — израсходованный интернет-трафик в Мб,

`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd # импорт библиотеки pandas
import numpy as np # импорт библиотеки numpy
import seaborn as sns # импорт библиотеки seaborn
import matplotlib.pyplot as plt #импорт библиотеки matplotlib.pyplot
from sklearn.model_selection import train_test_split # из библиотеки sklearn импортируем разделение выборки на тренировочную, тестовую и валидационную
from sklearn.linear_model import LogisticRegression # из библиотеки sklearn импортируем модель логистической регрессии
from sklearn.ensemble import RandomForestClassifier # из библиотеки sklearn импортируем модель случайного леса
from sklearn.tree import DecisionTreeClassifier # из библиотеки sklearn импортируем дерево решений для классификации данных
from sklearn.metrics import accuracy_score # из библиотеки sklearn импортируем вычесление доли правильных ответов
from sklearn.metrics import precision_score # из библиотеки sklearn импортируем значение точности
from sklearn.metrics import recall_score # из библиотеки sklearn импортируем значение полноты
from sklearn.metrics import f1_score # из библиотеки sklearn импортируем значение f1-меры
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning) #Скрываем лишние предупреждения

In [2]:
try:
    data = pd.read_csv('C:\\Data\\users_behavior.csv') #Добавили две \\ потому что кол выдавал ошибку
except: 
    data = pd.read_csv('/datasets/users_behavior.csv')

# Шаг 1. Изучение данных

In [3]:
data.info() #изучим данные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.shape # изучаем число строк и столбцов датафрейма

(3214, 5)

In [5]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [6]:
data.duplicated().sum() # явных дубликатов в столбцах нет

0

In [7]:
data.isna().sum() #нулевых значений в выборке нет

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [8]:
data['messages'] = data['messages'].astype(int) # изменим тип данных у строки messages, поскольку дробных значений у целого числа писем быть не может

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 125.7 KB


**Вывод по 1 шагу: Данные по столбцам корректны. Мы изменили тип у messages на целочисенный int. Пропусков и дубликатов нет.**

# Шаг 2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.

**Разобъём выборки на обучающую (60%), валидационную (20%) и тестовую (20%)**

In [10]:
data_train, data_other_sample = train_test_split(data, test_size=0.4, random_state=12345) # отделяем обучающую выборку от остальной
data_valid, data_test = train_test_split(data_other_sample, test_size=0.5, random_state=12345) # делим остальную часть выборки на валидационную и тренировочную

print('Размер обучающей выборки', data_train.shape[0])
print('Размер валидационной выборки', data_valid.shape[0])
print('Размер тестовой выборки', data_test.shape[0])

Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


**Составим признаки и целевой признак для каждоо датафрейма**

In [11]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

* features_train, tagret_train - обучающая выборка
* features_valid, target_valid - валидационная выборка
* features_test, target_test - тестовая выборка

**Вывод по 2 шагу: Мы разделили выборку по принципу 60/20/20, где 60% - обучающая выборка (data_train), 20% - валидационная выборка (data_valid), 20% - тестовая выборка (data_test).**

# Шаг 3. Исследование качества разных моделей, меняя гиперпараметры и описание кратких выводов исследования.

**Модель решающего дерева (Decision tree)**

In [12]:
%%time

best_depth = 0
best_accuracy = 0

for depth in range(1,101):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('Глубина дерева', depth, 'Качество модели', accuracy)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy
print('Лучшая глубина', best_depth, 'Лучшее качество модели:', best_accuracy)

Глубина дерева 1 Качество модели 0.7542768273716952
Глубина дерева 2 Качество модели 0.7822706065318819
Глубина дерева 3 Качество модели 0.7853810264385692
Глубина дерева 4 Качество модели 0.7791601866251944
Глубина дерева 5 Качество модели 0.7791601866251944
Глубина дерева 6 Качество модели 0.7838258164852255
Глубина дерева 7 Качество модели 0.7822706065318819
Глубина дерева 8 Качество модели 0.7791601866251944
Глубина дерева 9 Качество модели 0.7822706065318819
Глубина дерева 10 Качество модели 0.7744945567651633
Глубина дерева 11 Качество модели 0.7620528771384136
Глубина дерева 12 Качество модели 0.7620528771384136
Глубина дерева 13 Качество модели 0.7558320373250389
Глубина дерева 14 Качество модели 0.7589424572317263
Глубина дерева 15 Качество модели 0.7465007776049767
Глубина дерева 16 Качество модели 0.7340590979782271
Глубина дерева 17 Качество модели 0.7356143079315708
Глубина дерева 18 Качество модели 0.7309486780715396
Глубина дерева 19 Качество модели 0.7278382581648523
Гл

**Самая лучшая глубина дерева - 3.**

In [13]:
best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=3)
best_tree_model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Глубина дерева', 3, 'Лучшее качество модели', accuracy)

Глубина дерева 3 Лучшее качество модели 0.713841368584759


**Модель случайного леса (Random forest)**

In [14]:
%%time

best_estimation = 0
best_result = 0

for est in range(1, 101):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('Количество деревьев', est, 'Качество модели', accuracy)
    if accuracy > best_accuracy:
        best_estimation = est
        best_accuracy = accuracy
        
print('Лучшее число деревьев:', est, 'Лучшее качество модели', accuracy)

Количество деревьев 1 Качество модели 0.7107309486780715
Количество деревьев 2 Качество модели 0.7636080870917574
Количество деревьев 3 Качество модели 0.7387247278382582
Количество деревьев 4 Качество модели 0.7713841368584758
Количество деревьев 5 Качество модели 0.749611197511664
Количество деревьев 6 Качество модели 0.7807153965785381
Количество деревьев 7 Качество модели 0.7682737169517885
Количество деревьев 8 Качество модели 0.7822706065318819
Количество деревьев 9 Качество модели 0.7729393468118196
Количество деревьев 10 Качество модели 0.7853810264385692
Количество деревьев 11 Качество модели 0.7838258164852255
Количество деревьев 12 Качество модели 0.7869362363919129
Количество деревьев 13 Качество модели 0.7822706065318819
Количество деревьев 14 Качество модели 0.7838258164852255
Количество деревьев 15 Качество модели 0.7838258164852255
Количество деревьев 16 Качество модели 0.7869362363919129
Количество деревьев 17 Качество модели 0.7869362363919129
Количество деревьев 18 К

In [15]:
best_forest_model = RandomForestClassifier(random_state=12345, n_estimators=100)
best_forest_model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Количество деревьев', 100, 'Лучшее качество модели', accuracy)

Количество деревьев 100 Лучшее качество модели 0.7853810264385692


**Самая лучшее число деревьев - 100.**

**Модель логистической регрессии (Logistic regression)**

In [16]:
%%time

best_max_iter = 0
best_accuracy = 0

for max_iter in range(1, 201): 
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=max_iter)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('Количество итераций', max_iter, 'Качество модели:', accuracy)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy = accuracy
        
print('Лучшее число итераций:', best_max_iter, 'Лучшее качество модели:', best_accuracy)

Количество итераций 1 Качество модели: 0.7060653188180405
Количество итераций 2 Качество модели: 0.7060653188180405
Количество итераций 3 Качество модели: 0.7060653188180405
Количество итераций 4 Качество модели: 0.7060653188180405
Количество итераций 5 Качество модели: 0.7060653188180405
Количество итераций 6 Качество модели: 0.7060653188180405
Количество итераций 7 Качество модели: 0.7060653188180405
Количество итераций 8 Качество модели: 0.7060653188180405
Количество итераций 9 Качество модели: 0.7060653188180405
Количество итераций 10 Качество модели: 0.7060653188180405
Количество итераций 11 Качество модели: 0.7060653188180405
Количество итераций 12 Качество модели: 0.7060653188180405
Количество итераций 13 Качество модели: 0.7060653188180405
Количество итераций 14 Качество модели: 0.7060653188180405
Количество итераций 15 Качество модели: 0.7060653188180405
Количество итераций 16 Качество модели: 0.7060653188180405
Количество итераций 17 Качество модели: 0.7060653188180405
Количе

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Количество итераций 19 Качество модели: 0.7076205287713841
Количество итераций 20 Качество модели: 0.7076205287713841
Количество итераций 21 Качество модели: 0.7076205287713841
Количество итераций 22 Качество модели: 0.7076205287713841
Количество итераций 23 Качество модели: 0.7076205287713841
Количество итераций 24 Качество модели: 0.7076205287713841
Количество итераций 25 Качество модели: 0.7076205287713841
Количество итераций 26 Качество модели: 0.7076205287713841
Количество итераций 27 Качество модели: 0.7076205287713841
Количество итераций 28 Качество модели: 0.7076205287713841
Количество итераций 29 Качество модели: 0.7076205287713841
Количество итераций 30 Качество модели: 0.7091757387247278


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Количество итераций 31 Качество модели: 0.7091757387247278
Количество итераций 32 Качество модели: 0.7076205287713841
Количество итераций 33 Качество модели: 0.7076205287713841
Количество итераций 34 Качество модели: 0.7076205287713841
Количество итераций 35 Качество модели: 0.7076205287713841
Количество итераций 36 Качество модели: 0.7060653188180405
Количество итераций 37 Качество модели: 0.7045101088646968
Количество итераций 38 Качество модели: 0.7091757387247278


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Количество итераций 39 Качество модели: 0.7107309486780715
Количество итераций 40 Качество модели: 0.7107309486780715
Количество итераций 41 Качество модели: 0.7107309486780715
Количество итераций 42 Качество модели: 0.7107309486780715
Количество итераций 43 Качество модели: 0.7107309486780715
Количество итераций 44 Качество модели: 0.7107309486780715
Количество итераций 45 Качество модели: 0.7107309486780715
Количество итераций 46 Качество модели: 0.7107309486780715
Количество итераций 47 Качество модели: 0.7107309486780715
Количество итераций 48 Качество модели: 0.7107309486780715


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Количество итераций 49 Качество модели: 0.7107309486780715
Количество итераций 50 Качество модели: 0.7107309486780715
Количество итераций 51 Качество модели: 0.7107309486780715
Количество итераций 52 Качество модели: 0.7107309486780715
Количество итераций 53 Качество модели: 0.7107309486780715
Количество итераций 54 Качество модели: 0.7107309486780715
Количество итераций 55 Качество модели: 0.7107309486780715
Количество итераций 56 Качество модели: 0.7107309486780715
Количество итераций 57 Качество модели: 0.7107309486780715
Количество итераций 58 Качество модели: 0.7107309486780715
Количество итераций 59 Качество модели: 0.7107309486780715
Количество итераций 60 Качество модели: 0.7107309486780715
Количество итераций 61 Качество модели: 0.7107309486780715
Количество итераций 62 Качество модели: 0.7107309486780715
Количество итераций 63 Качество модели: 0.7107309486780715
Количество итераций 64 Качество модели: 0.7107309486780715
Количество итераций 65 Качество модели: 0.71073094867807

In [17]:
best_regression_model = LogisticRegression(random_state=12345)
best_regression_model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Количество итераций', 39, 'Лучшее качество модели:', accuracy)

Количество итераций 39 Лучшее качество модели: 0.7107309486780715


**Лучшее количество итераций - 39.**

**Вывод по 3 шагу:**

* Качаство модели дерева решений (Descision tree) при глубине 3 равна Время выполнения равна 0.78.
* Качаство модели случайного леса (Random forest) при количестве деревьев 100 равна 0.78.
* Качаство модели логистической регрессии (Logistic regression) при количество итераций 39 равна 0.71.

Поскольку выборка небольшая, то лучший результат показывают модели дерева решений и модель случайного леса.

# Шаг 4. Проверить качество модели на тестовой выборке.

In [18]:
test_predictions = best_tree_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Качество модели решающего дерева на тестовой выборке', accuracy)

test_predictions = best_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Качество модели случайного леса на тестовой выборке', accuracy)

test_predictions = accuracy_score(target_test, test_predictions)
print('Качество модели логистической регрессии на тестовой выборке', accuracy)

Качество модели решающего дерева на тестовой выборке 0.7791601866251944
Качество модели случайного леса на тестовой выборке 0.7853810264385692
Качество модели логистической регрессии на тестовой выборке 0.7853810264385692


**Вывод по 4 шагу: Чтобы решить поставленную задачу, необходимо выбрать модель случайного леса, либо логистической регрессии. Для гарантии лучше использовать модель случайного леса, поскольку в обоих случаях моедь показала одинаково высокие результаты**

# Шаг 5. Дополнительное задание: проверить модели на вменяемость.

Для проверки модели на адекватность будем использовать три метрики: 

* `accuracy` - качество (Отношение числа правильных ответов к размеру тестовой выборки)
* `precision` - точность (это метрика качества классификации, которая показывает какую долю объектов, распознанных как объекты положительного класса, мы предсказали верно. Вычисляется по формуле: TP / (TP + FP)).
* `recall` - полнота (это метрика качества классификации, показывающая, какова доля TP-ответов среди всех, у которых истинная метка 1. Вычисляется по формуле:TP / (TP + FN)).

И F1-меру:

* `F1-мера` - это среднее гармоническое полноты и точности. Единица в F1 означает, что соотношение полноты и точности равно 1:1.

In [19]:
test_predictions = best_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
precision = precision_score(target_test, test_predictions)
recall = recall_score(target_test, test_predictions)
print('Accuracy', accuracy, 'Precision:', precision, 'Recall:', recall)

Accuracy 0.7853810264385692 Precision: 0.696969696969697 Recall: 0.5665024630541872


In [20]:
f_score = f1_score(target_test, test_predictions)
print('F1-мера:', f_score)

F1-мера: 0.6250000000000001


**Вывод по 5 шагу: Исходя из вычесления вышеперечисленных метрик `accuracy`, `precision`, `recall` и `F1-меры`, можно сказать, что модель получилась среднего качества.**

In [21]:
from sklearn.dummy import DummyClassifier

In [22]:
frequent_clf = DummyClassifier(strategy='most_frequent').fit(features_train, target_train) #метод прогнозирования всегда возвращает наиболее часто встречающуюся метку класса в наблюдаемом аргументе target, переданном для fit.

In [23]:
target_train_prediction = frequent_clf.predict(features_test)

In [24]:
print('Unique predicted labels: ', (np.unique(target_train_prediction)))

Unique predicted labels:  [0]


In [25]:
print('Test score: ', accuracy_score(target_test, target_train_prediction))

Test score:  0.6842923794712286


In [26]:
uniform_clf = DummyClassifier(strategy='uniform').fit(features_train, target_train)

In [27]:
target_uniform_pred = uniform_clf.predict(features_test)

In [28]:
frequent_clf.score(features_test, target_test)

0.6842923794712286

In [29]:
uniform_clf.score(features_test, target_test) #генерирует предсказания равномерно случайным образом из списка уникальных классов, наблюдаемых в target, т.е. каждый класс имеет равную вероятность.

0.4774494556765163

In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(target_test, predictions)

array([[432,   8],
       [194,   9]])

In [32]:
confusion_matrix(target_test, target_uniform_pred)

array([[223, 217],
       [116,  87]])

In [33]:
confusion_matrix(target_test, target_train_prediction) 

array([[440,   0],
       [203,   0]])

# Общий вывод:

В ходе проделанной работы мы выполнили следующие шаги:

* Шаг 1. Изучили датафрейм.
* Шаг 2. Разбили выборку на обучающую, валидационную и тестовую по соотношению 60/20/20.
* Шаг 3. Исследовали три модели: Решающее дерево (Decision tree), случайный лес (Random forest) и логистическую регрессию (Logistic regression). Выбрали наиболее подходящую модель для исследования и обучили её - ей оказалась модель случайного леса (Random forest) со значенем 100 деревьев и качеством `accuracy` 0.78.
* Шаг 4. Оценили качество обученной модели на тестовой выборке с помощью метрики `accuracy`.
* Шаг 5. Проверили модель на вменяемость с помощью метрик: `accuracy`, `precision`, `recall` и `F1-меры`.

Для обучения модели мы использовали 1928 ответов, что недостаточно для обучения. необходимо увеличить выборку в несколько раз, чтобы повысить метрики `accuracy`, `precision`, `recall` и `F1-меру`.

Исходя из вычесления вышеперечисленных метрик accuracy, precision, recall и F1-меры, можно сказать, что модель получилась среднего качества - 0.63.